In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-tj2iau3r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-tj2iau3r
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━

In [23]:
import torch
import whisper
import numpy as np
from whisper.tokenizer import get_tokenizer  # tokenizer를 별도로 불러옵니다.

# soft capping 함수: logit 값이 cap 이상일 때 부드럽게 포화시키는 함수
def soft_cap_logits(logits, cap=10.0):
    # logits에 대해 tanh 함수를 적용하여 부드러운 상한선을 설정합니다.
    return cap * torch.tanh(logits / cap)
# Whisper 모델 로드 (예: "base" 모델)
version = "turbo"
model = whisper.load_model(version)
device = model.device

# 오디오 파일 로드 및 전처리 (예: 'audio_file.wav' 경로의 파일)
audio_path = "audio.mp3"  # Colab에 업로드한 오디오 파일 경로로 변경하세요.
audio = whisper.load_audio(audio_path)
audio = whisper.pad_or_trim(audio)
if version in ["base"]:
    # Log-Mel 스펙트로그램 계산 후 모델의 디바이스로 이동
    mel = whisper.log_mel_spectrogram(audio).to(device)
    # 인코더 실행: 오디오를 인코딩하여 encoder_out 획득
    encoder_out = model.encoder(mel.unsqueeze(0))
else:
    encoder_out = model.encoder(torch.tensor(audio).unsqueeze(0).to(device))

# 디코딩 설정: 최대 길이, 시작 토큰 등 설정 (실험용으로 간단히 greedy decoding 진행)
max_length = 50  # 최대 생성 길이
# sot_token = model.tokenizer.sot_token  # 시작 토큰 (start-of-transcript)
# eot_token = model.tokenizer.eot_token  # 종료 토큰 (end-of-transcript)
tokenizer = get_tokenizer(language="ko", task="transcribe", multilingual=True)
sot_token = tokenizer.sot  # 시작 토큰
eot_token = tokenizer.eot  # 종료 토큰

# 디코딩 루프 초기화: 시작 토큰으로 디코딩 시작
decoder_input = torch.tensor([[sot_token]], device=device)
decoded_tokens = [sot_token]

# greedy decoding 루프 (soft capping을 적용)
for _ in range(max_length):
    # 디코더에 현재 입력과 encoder_out을 넣어 logits 계산
    # logits의 shape: (batch, seq_len, vocab_size)
    logits = model.decoder(decoder_input, encoder_out)

    # 마지막 시간 스텝의 logits 추출
    next_token_logits = logits[0, -1, :]

    # soft capping 적용
    capped_logits = soft_cap_logits(next_token_logits, cap=5.0)

    # greedy 방식: 가장 높은 값을 갖는 토큰 선택
    next_token = torch.argmax(capped_logits).unsqueeze(0).unsqueeze(0)

    # 선택된 토큰을 입력에 추가
    decoder_input = torch.cat([decoder_input, next_token], dim=1)
    token = next_token.item()
    decoded_tokens.append(token)

    # 종료 토큰이면 루프 종료
    if token == eot_token:
        break

# 토큰을 텍스트로 변환 (특수 토큰은 생략)
transcription = tokenizer.decode(decoded_tokens)
print("Transcription with soft capping applied:")
print(transcription)


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


RuntimeError: Given groups=1, weight of size [1280, 128, 3], expected input[1, 1, 480000] to have 128 channels, but got 1 channels instead